In [3]:
import os
from openai import OpenAI

# Call OpenAI_API

In [4]:
client = OpenAI()

In [3]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role" : "user", "content" : "is it too late now?"}]
)

In [4]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The answer to your question really depends on the context. "Too late" can refer to a variety of situations, such as meeting a deadline, making a decision, taking a specific action, or addressing an issue. Can you provide more details or specify the situation you\'re referring to? That way, I can offer a more precise and helpful answer.', role='assistant', function_call=None, tool_calls=None))

# Simple RAG

In [5]:
import minsearch
import json

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Index the documents

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [10]:
index.fit(documents)

## Retrieve the relevant answer to the user's questionfrom the knowledge base

In [11]:
q = 'the course has already started, can I still enroll?'

In [12]:
boost = {'question' : 3.0, 'section' : 0.5, } # means that the section QUESTION from text_field is 3 times more important than TEXT and SECTION is half important from TEXT

results = index.search(
    query = q,
    filter_dict={'course' : 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results = 5
    )

In [13]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

##  Send documents from Knowledge Base to LLM as a context to the user's question.

In [14]:
q

'the course has already started, can I still enroll?'

In [15]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role" : "user", "content" : q}]
)

In [16]:
response.choices[0].message.content

"It depends on the specific course and the policies of the institution offering it. Some courses allow late enrollment, especially within the first few weeks, while others have strict deadlines. Here are a few steps you can take:\n\n1. **Check the Course Information:** Review the course syllabus or description to see if there are any mentions of late enrollment policies.\n\n2. **Contact the Instructor:** Reach out to the course instructor or professor. They may allow you to join the class if you explain your situation and demonstrate your commitment.\n\n3. **Speak with the Registrar or Academic Advisor:** The registrar's office or your academic advisor can provide detailed information about the enrollment policies and any possible options for late joining.\n\n4. **Review the Institution's Policies:** Some schools have a formal add/drop period during which you are allowed to make changes to your schedule.\n\n5. **Consider the Workload:** If the course has already started, you’ll need to

## Clean code for RAG

In [20]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [19]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [21]:
rag(query)

"To run Kafka, follow the steps relevant to your development context:\n\n### For Java:\nIn the project directory, use the following command to run a Kafka Java application (producer, consumer, kstreams, etc.):\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\nReplace `<jar_name>` with the actual name of your JAR file.\n\n### For Python:\n1. **Create a virtual environment** (required only once):\n   ```sh\n   python -m venv env\n   source env/bin/activate\n   pip install -r ../requirements.txt\n   ```\n   On Windows, activate the virtual environment with:\n   ```sh\n   env\\Scripts\\activate\n   ```\n\n2. **Activate the virtual environment** (required every time you run the Python file):\n   ```sh\n   source env/bin/activate\n   ```\n   On Windows:\n   ```sh\n   env\\Scripts\\activate\n   ```\n\n3. **Deactivate the virtual environment** (when done):\n   ```sh\n   deactivate\n   ```\n   \nMake sure Docker images are running if you

# RAG using ElasticSearch

In [22]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [27]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
es_client = Elasticsearch('http://localhost:9200')

In [25]:
es_client.info()

ObjectApiResponse({'name': '9ab30dd498cd', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'W-Q1lZ1URO-OLgKpxafpbg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [28]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████| 948/948 [00:31<00:00, 30.11it/s]


In [29]:
query = 'I just disovered the course. Can I still join it?'

In [34]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }


In [40]:
response = es_client.search(index=index_name, body=search_query)
response

ObjectApiResponse({'took': 10, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 405, 'relation': 'eq'}, 'max_score': 72.849266, 'hits': [{'_index': 'course-questions', '_id': '2p0mW5AB3Lcv1_TTlFJx', '_score': 72.849266, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': '350mW5AB3Lcv1_TTlVIP', '_score': 54.057133, '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next 

In [41]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': '2p0mW5AB3Lcv1_TTlFJx',
  '_score': 72.849266,
  '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
   'section': 'General course-related questions',
   'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '350mW5AB3Lcv1_TTlVIP',
  '_score': 54.057133,
  '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
   'section': 'General course-related questions',
   'question': 'Course - Can I follow the course after 

In [37]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [38]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [39]:
rag(query)

'Yes, you can still join the course even if you discovered it after the start date. You are eligible to submit the homework and access the materials. Just be aware of the deadlines for the final projects and try not to leave everything until the last minute.'